In [ ]:
# Setup the standalone version of Dash
from dash import Dash

# Configure the necessary Python module imports for dashboard components
from dash import html, dcc, Input, Output     
import dash_leaflet as dl                           
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# used for fuzzy matching
import difflib
import plotly.express as px
from dash import dash_table
from sqlalchemy import create_engine

#from CRUD import AnimalShelter


###########################
# Data Manipulation / Model
###########################

# hard coded credentials for the mySQL connection
username = "aacuser"
password = "SNHU1234"
host = "localhost"
db = "rescues"

engine =  create_engine(f"mysql+pymysql://{username}:{password}@{host}/{db}")

df = pd.read_sql_table("animals", engine)


# error reading in the age in weeks, so this calculation is a work around
def convert_age_to_weeks(age_str):
    if not isinstance(age_str, str) or age_str.strip() == "":
        return None
    try:
        num, unit = age_str.strip().lower().split()
        num = int(num)
        if 'year' in unit:
            return num * 52
        elif 'month' in unit:
            return num * 4
        elif 'week' in unit:
            return num
        elif 'day' in unit:
            return max(1, num // 7)
    except Exception as e:
        return None

df['age_upon_outcome_in_weeks'] = df['age_upon_outcome'].apply(convert_age_to_weeks)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


################################
# Precompute Rescue type buckets
#################################
precomputed = {
    'Reset': df, # reset returns all data
    'Water Rescue': df[ # water rescue returns animals that would be ideal for this
        df.breed.isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])
            & (df.sex_upon_outcome == 'Intact Female')
            & ((df.age_upon_outcome_in_weeks >= 26) & (df.age_upon_outcome_in_weeks <= 156))
    ],
    'Mountain Rescue':  df[ # mountain rescue returns animals that would be ideal for this
        df.breed.isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'])
            & (df.sex_upon_outcome == 'Intact Male')
            & ((df.age_upon_outcome_in_weeks >= 26) & (df.age_upon_outcome_in_weeks <= 156))
    ],
    'Disaster Rescue': df[ # disaster rescue returns animals that would be ideal for this
        df.breed.isin(['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'])
            & (df.sex_upon_outcome == 'Intact Male')
            & ((df.age_upon_outcome_in_weeks >= 20) & (df.age_upon_outcome_in_weeks <= 300))
    ]
}

#########################
# Dashboard Layout / View
#########################
app = Dash('Project 2')

#Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # added logo as requested
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        [
        html.A(
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
            # these open in jupyter because dash doesn't allow for opening in different browser, so I commented them out
            #href = "www.snhu.edu",
            #target = "_blank"
        ),
        html.B(html.H3('App By Brandon Hollada')),# added name as identifier
        ],
        style={'textAlign': 'center'}
    ),   
#added radio buttons to select filters based on Grazioso Salvare criteria for each rescue animal
    html.Div(
        dcc.RadioItems(
            id = 'filter-type',
            options = [{'label': k, 'value': k} for k in precomputed.keys()],
            value = 'Reset', #set initial radio button to reset as it retrieves all data
            inline=True,
            style={'display': 'flex', # makes the inputs inline with spacing
                    'justifyContent': 'center',
                    'gap': '20px'}
        ),
        style={'textAlign': 'center', # centers the group
                'padding': '10px 0'}
    ),

    # adds a fuzzy search 
    html.Div([
        dcc.Input(
            id="search-term",
            type="text",
            placeholder="Search breed or age",
            style={"width": "50%", "padding": "5px"}
        )
    ]),

    html.Hr(),
    # set filter options and formatiing for the data table to be user friendly
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True}
                                  for i in df.columns],
                         data=df.to_dict('records'),
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable=False,
                         row_selectable="single",
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[0],
                         page_action="native",
                         page_current=0,
                         page_size=10
                        ),
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

    
@app.callback([Output('datatable-id','data'),
              Output('datatable-id','columns')],
              [Input('filter-type', 'value'),
              Input('search-term', 'value')])
def update_dashboard(filter_type, search_value):
## Used if statement to filter the data table based on which radio button is selected
    dff = precomputed.get(filter_type, df).copy()
       
    if dff.empty:# this catches empty dataframes to prevent errors
        return [], [{'name': c, 'id': c} for c in df.columns]

    if search_value:
        search = search_value.strip()
        if '-' in search:
            low, high = search.split('-', 1)
            if low.strip().isdigit() and high.strip().isdigit():
                low, high = int(low), int(high)
                dff = dff[
                    (dff['age_upon_outcome_in_weeks'] >= low) &
                    (dff['age_upon_outcome_in_weeks'] <= high)
                ]
        #checks if value is a number for age
        elif search.isdigit():
            goal = int(search)
            max_age = dff['age_upon_outcome_in_weeks'].max() or 1
            dff['match_score'] = 1 - ((dff['age_upon_outcome_in_weeks'] - goal).abs() / max_age)
            dff = ( dff.sort_values('match_score', ascending=False).drop(columns= ['match_score']))
        else:
            searchLower = search.lower()
            dff['match_score'] = dff['breed'].apply(    
                lambda b: difflib.SequenceMatcher(None, b.lower(), searchLower).ratio()
            )
            dff = ( dff.sort_values('match_score', ascending=False).drop(columns=['match_score']))

    return dff.to_dict('records'), [{'name': c, 'id': c} for c in dff.columns]

# Display the breeds of animal based on quantity represented in
# the data table
# update to not display if search data is incorrect
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
    Input('filter-type', 'value')])
def update_graphs(viewData, filter_type):
    #checks if data is empty
    if not viewData:
        return []

    dff = pd.DataFrame(viewData)
    if dff.empty:
        return []    

    pie_col = 'animal_type' if filter_type == 'Reset' else 'breed'

    fig = px.pie(dff, names=pie_col, title='Preferred Animals')

    return [dcc.Graph(figure=fig)]    
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None or len(viewData) == 0: # added check for 0 to prevent errors
        return
    elif index is None or len(index) == 0: # added check for 0 to prevent errors
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run(debug=True, port=8050)
